Imports

In [1]:
from google.colab import drive
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

ModuleNotFoundError: ignored

Mount drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
def init_webpage(url):
    driver = webdriver.Chrome()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return driver, soup

Parse Talk Corpus (https://www.researchgate.net/publication/338691522_Talk_Corpus_A_Web-based_Corpus_of_TED_Talks_for_English_Language_Teachers_and_Learners)

In [ ]:
images_location = '/content/drive/MyDrive/SSL&NLP/data/talkcorpus'

url = 'http://talkcorpus.com/'
driver, soup = init_webpage(url)

talk_titles = soup.find_all('tr', {'class': 'talk-link'})
metrics = ['WPM', 'Length', 'NAWL', 'NGSL']  # from td blocks without title attribute

# loop through each talk title and extract the metrics and url
for talk_title in talk_titles:
    data_id = talk_title.get('data-id')
    data_title = talk_title.find('td', {'title': data_id}).text
    metadata = {'id': data_id, 'title': data_title}
    metrics_data = []
    for td in talk_title.contents:
        if td.has_attr('title'):
            if td.get('title') != data_id:  # talk title and id have already been added
                metadata['FKRE_rating'] = td.get('title')
                metadata['FKRE_score'] = td.text
        else:
            metrics_data.append(td.text)
    for i in range(len(metrics)):
        metadata[metrics[i]] = metrics_data[i]

    # make sure talk data panel corresponds to the current talk
    talk = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, f"tr[data-id='{data_id}']")))
    talk.click()

    # click dropdown to make menu visible to driver
    dropdown = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#talk-details .dropdown-toggle")))
    dropdown.click()

    talk_url = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#talk-details a[target='_blank']"))).get_attribute("href")
    metadata['URL'] = talk_url

    filename = f'{data_id}_{data_title}.json'
    filename = filename.replace(": ", "_")  # make sure name is valid for any OS
    with open(filename, 'w') as outfile:
        json.dump(metadata, outfile, indent=4)

driver.quit()